In [8]:
import numpy as np
import torch
import torch.nn.functional as F
from prep import get_data
from torch import nn
from kfold_experiments import kfold_normal_training_softgedt, kfold_adversarial_training_softgedt,kfold_normal_training_neural_networks, kfold_adversarial_training_neural_networks
import create_file_folder

# Create dataset

In [9]:
# load dataset
filename = 'vehicle'
create_file_folder(filename)

# Preprocess the data without standardizing
data, ncat,dtypes = get_data(filename)

# create editability vector
non_editable_vector = torch.tensor([1 for i in range(data[:,:-1].shape[1])])

# record datatypes
dtypes[dtypes=='object'] = 'pos_int'
dtypes[dtypes=='float64'] = 'float'

datatypes = dtypes[:-1].to_numpy().astype('object')
datatypes[(datatypes=='int64') & np.min(data[:,:-1]>=0,axis=0)]='pos_int'
datatypes[(datatypes=='float64') & np.min(data[:,:-1]>=0,axis=0)]='pos_float'
datatypes = datatypes.tolist()

# Normal Training

In [10]:
kfold_normal_training_softgedt(filename, data, ncat, non_editable_vector, datatypes, include_sum_weight = True)

experiment: 0
Fold: 0
epoch: 0
train acc: tensor(0.8560) valid acc: tensor(0.8047) trigger times: 0
epoch: 1
train acc: tensor(0.8994) valid acc: tensor(0.7988) trigger times: 0
epoch: 2
train acc: tensor(0.9034) valid acc: tensor(0.8047) trigger times: 1
epoch: 3
train acc: tensor(0.9034) valid acc: tensor(0.7988) trigger times: 0
epoch: 4
train acc: tensor(0.9152) valid acc: tensor(0.8107) trigger times: 1
epoch: 5
train acc: tensor(0.9270) valid acc: tensor(0.8166) trigger times: 0
epoch: 6
train acc: tensor(0.9290) valid acc: tensor(0.8047) trigger times: 0
epoch: 7
train acc: tensor(0.9270) valid acc: tensor(0.8107) trigger times: 1
epoch: 8
train acc: tensor(0.9428) valid acc: tensor(0.7929) trigger times: 0
epoch: 9
train acc: tensor(0.9310) valid acc: tensor(0.7988) trigger times: 1
Low density samples found at indices  tensor([ 11,  73, 132]) . Probably out-of-domain samples.
SoftGeDT Accuracy on Training set: tensor(0.9310)
SoftGeDT Accuracy on Validation set: tensor(0.7988)


# Adversarial Training

In [3]:
kfold_adversarial_training_softgedt(filename, data, ncat, non_editable_vector, datatypes, include_sum_weight=True)

Fold: 0
epoch: 0
train error: 0.571992110453649 train loss: 2.9393895015678932 validation error 0.5857988165680473
epoch: 1
train error: 0.5522682445759369 train loss: 1.7257483969543577 validation error 0.5562130177514792
epoch: 2
train error: 0.504930966469428 train loss: 1.5423431697446683 validation error 0.47337278106508873
epoch: 3
train error: 0.45956607495069035 train loss: 1.3208749308388614 validation error 0.5029585798816568
epoch: 4
train error: 0.4516765285996055 train loss: 1.1893272588003787 validation error 0.4970414201183432
epoch: 5
train error: 0.46153846153846156 train loss: 1.115833395331569 validation error 0.5266272189349113
epoch: 6
train error: 0.46745562130177515 train loss: 1.1161402489776913 validation error 0.4911242603550296
epoch: 7
train error: 0.4418145956607495 train loss: 1.085714554645606 validation error 0.46153846153846156
epoch: 8
train error: 0.4240631163708087 train loss: 1.0796853353285931 validation error 0.4378698224852071
epoch: 9
train erro

# Simple Neural Network

In [3]:
class Net(nn.Module):
    def __init__(self, input_dim):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(input_dim, 80).double()
        self.layer2 = nn.Linear(80, 80).double()
        self.layer3 = nn.Linear(80, 80).double()
        self.layer4 = nn.Linear(80, 4).double()
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x))
        x = self.layer4(x)
        return x

## 5-fold cross validation normal training

In [7]:
kfold_normal_training_neural_networks(nn, filename, data, ncat, non_editable_vector, datatypes)

Fold: 0
epoch: 0
train acc: tensor(0.4990) valid acc: tensor(0.6036) trigger times: 0
epoch: 1
train acc: tensor(0.6864) valid acc: tensor(0.7160) trigger times: 0
epoch: 2
train acc: tensor(0.7337) valid acc: tensor(0.8225) trigger times: 0
epoch: 3
train acc: tensor(0.7633) valid acc: tensor(0.7160) trigger times: 0
epoch: 4
train acc: tensor(0.7909) valid acc: tensor(0.7988) trigger times: 1
epoch: 5
train acc: tensor(0.7929) valid acc: tensor(0.8107) trigger times: 0
epoch: 6
train acc: tensor(0.8126) valid acc: tensor(0.8047) trigger times: 0
epoch: 7
train acc: tensor(0.7890) valid acc: tensor(0.7219) trigger times: 1
epoch: 8
train acc: tensor(0.8323) valid acc: tensor(0.8166) trigger times: 2
epoch: 9
train acc: tensor(0.8462) valid acc: tensor(0.8047) trigger times: 0
epoch: 10
train acc: tensor(0.8402) valid acc: tensor(0.7988) trigger times: 1
epoch: 11
train acc: tensor(0.8501) valid acc: tensor(0.8107) trigger times: 2
epoch: 12
train acc: tensor(0.8521) valid acc: tensor(

## 5 fold cross validation adversarial training

In [8]:
kfold_adversarial_training_neural_networks(Net, filename, data, ncat, non_editable_vector, datatypes)

Fold: 0
epoch: 0
train error: 0.5897435897435898 train loss: 1.2667927077630683 validation error 0.4319526627218935
epoch: 1
train error: 0.4930966469428008 train loss: 1.0806504490296391 validation error 0.46153846153846156
epoch: 2
train error: 0.4477317554240631 train loss: 0.9586642923823173 validation error 0.48520710059171596
epoch: 3
train error: 0.42998027613412226 train loss: 0.8947408202752205 validation error 0.40236686390532544
epoch: 4
train error: 0.41025641025641024 train loss: 0.8364671493935595 validation error 0.4556213017751479
epoch: 5
train error: 0.39644970414201186 train loss: 0.8403812292278803 validation error 0.38461538461538464
epoch: 6
train error: 0.388560157790927 train loss: 0.823972435675736 validation error 0.44970414201183434
epoch: 7
train error: 0.40039447731755423 train loss: 0.8016286989997052 validation error 0.42011834319526625
epoch: 8
train error: 0.40631163708086787 train loss: 0.8109585159402333 validation error 0.4556213017751479
epoch: 9
tr